In [8]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from scipy.stats import randint

# 📌 1. Veri Ön İşleme Fonksiyonu
def preprocess_data(file_path, is_train=True):
    df = pd.read_csv(file_path)
    print(f"✅ Veri yüklendi: {df.shape}")

    # 📌 Sayısal ve Kategorik Sütunları Belirleme
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

    print(f"🔹 Kategorik Sütunlar: {categorical_cols}")
    print(f"🔹 Sayısal Sütunlar: {numerical_cols}")

    # 📌 Eksik değerleri doldurma
    num_imputer = SimpleImputer(strategy="median")
    cat_imputer = SimpleImputer(strategy="most_frequent")

    if numerical_cols:
        df[numerical_cols] = num_imputer.fit_transform(df[numerical_cols])

    if categorical_cols:
        df[categorical_cols] = cat_imputer.fit_transform(df[categorical_cols].astype(str))  

    print("✅ Eksik değerler dolduruldu!")

    # 📌 Kategorik değişkenleri encode et
    for col in categorical_cols:
        df[col] = LabelEncoder().fit_transform(df[col])

    print("✅ Kategorik değişkenler encode edildi!")

    # 📌 Train seti için hedef değişkeni ayır
    if is_train:
        y = df["rainfall"]
        X = df.drop(columns=["rainfall", "id"])
        return X, y
    else:
        test_ids = df["id"]
        X = df.drop(columns=["id"])
        return X, test_ids

# 📌 2. CatBoost Modelini Eğitme
def train_catboost(X_train, y_train):
    param_grid = {
        "iterations": randint(100, 1000),
        "learning_rate": [0.01, 0.02, 0.05, 0.1],
        "depth": randint(3, 10),
        "l2_leaf_reg": randint(1, 10),
        "border_count": randint(32, 255)
    }

    cat_model = CatBoostClassifier(
        loss_function="Logloss",
        eval_metric="AUC",
        random_seed=42,
        verbose=100,
        task_type="CPU"  # ✅ GPU yerine CPU kullanıyoruz!
    )

    search = RandomizedSearchCV(cat_model, param_grid, n_iter=15, scoring="roc_auc", cv=3, verbose=1, n_jobs=-1, random_state=42)
    search.fit(X_train, y_train)

    best_model = search.best_estimator_
    print(f"✅ En iyi CatBoost Modeli bulundu: {search.best_params_}")
    return best_model

# 📌 3. Test Verisinde Tahmin Yapma ve Sonucu Kaydetme
def predict_and_save(model, test_file, output_file="submission_catboost.csv"):
    X_test, test_ids = preprocess_data(test_file, is_train=False)
    y_pred = model.predict_proba(X_test)[:, 1]  # Yağmur olma olasılığını al
    submission = pd.DataFrame({"id": test_ids, "rainfall": y_pred})
    submission.to_csv(output_file, index=False)
    print(f"✅ Tahminler {output_file} dosyasına kaydedildi!")

# 📌 Ana Akış
if __name__ == "__main__":
    train_file = "train.csv"
    test_file = "test.csv"

    # 📌 Veriyi yükle ve işle
    X_train, y_train = preprocess_data(train_file, is_train=True)

    # 📌 Modeli eğit
    best_catboost_model = train_catboost(X_train, y_train)

    # 📌 Test verisi üzerinde tahmin yap ve sonucu kaydet
    predict_and_save(best_catboost_model, test_file)


✅ Veri yüklendi: (2190, 13)
🔹 Kategorik Sütunlar: []
🔹 Sayısal Sütunlar: ['id', 'day', 'pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint', 'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed', 'rainfall']
✅ Eksik değerler dolduruldu!
✅ Kategorik değişkenler encode edildi!
Fitting 3 folds for each of 15 candidates, totalling 45 fits
0:	total: 148ms	remaining: 51.9s
100:	total: 583ms	remaining: 1.45s
200:	total: 1.03s	remaining: 774ms
300:	total: 1.47s	remaining: 248ms
351:	total: 1.69s	remaining: 0us
✅ En iyi CatBoost Modeli bulundu: {'border_count': 89, 'depth': 8, 'iterations': 352, 'l2_leaf_reg': 9, 'learning_rate': 0.01}
✅ Veri yüklendi: (730, 12)
🔹 Kategorik Sütunlar: []
🔹 Sayısal Sütunlar: ['id', 'day', 'pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint', 'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed']
✅ Eksik değerler dolduruldu!
✅ Kategorik değişkenler encode edildi!
✅ Tahminler submission_catboost.csv dosyasına kaydedildi!
